<a href="https://colab.research.google.com/github/nnilayy/MedGPT/blob/main/distributed_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate datasets evaluate transformers

In [ ]:
import torch
from evaluate import load
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from accelerate import Accelerator
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def compute_metric(predictions, references, total_loss, loader, metric):
    metrics_result = metric.compute(predictions=predictions, references=references)
    metrics = metrics_result['accuracy']
    loss = total_loss / len(loader)
    return metrics, loss


def dataset(dataset_name, batch_size, tokenizer):
    dataset = load_dataset(dataset_name, 'mrpc')

    def encode(examples):
        return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length', max_length=128)

    dataset = dataset.map(encode, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    train_dataset = dataset['train']
    eval_dataset = dataset['validation']

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=data_collator)

    return train_loader, eval_loader


def train(epoch, model, loader, optimizer, accelerator, metric):
    model.train()
    total_loss = 0
    all_predictions = []
    all_references = []
    loop = tqdm(loader, desc=f"Training Epoch {epoch}", disable=not accelerator.is_local_main_process)
    for batch in loop:
        outputs = model(**batch)
        loss = outputs.loss
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        references = batch['labels'].detach().cpu().numpy()
        all_predictions.extend(predictions)
        all_references.extend(references)

    # Compute accuracy using evaluate
    train_accuracy, train_loss = compute_metric(all_predictions, all_references, total_loss, loader, metric)
    return train_accuracy, train_loss



# Evaluation function without tqdm
def evaluate(model, loader, metric):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_references = []
    for batch in loader:
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
            loss = outputs.loss
            total_loss += loss.item()

            predictions = torch.argmax(logits, dim=-1).detach().cpu().numpy()
            references = batch['labels'].detach().cpu().numpy()
            all_predictions.extend(predictions)
            all_references.extend(references)

    # Compute accuracy using evaluate
    valid_accuracy, valid_loss = compute_metric(all_predictions, all_references, total_loss, loader, metric)
    return valid_accuracy, valid_loss


def main():
# Initiate Accelerator
    accelerator = Accelerator()

# Download Model and Tokenizer
    checkpoint = 'bert-base-uncased'
    model = BertForSequenceClassification.from_pretrained(checkpoint, force_download=True, num_labels=2)
    tokenizer = BertTokenizer.from_pretrained(checkpoint, force_download=True)

# Setting up Hyperparameters
    epochs = 10
    lr = 5e-4
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    batch_size = 128
    metric = load("accuracy")

# Processing Dataset
    dataset_name = 'glue'
    train_loader, eval_loader = dataset(dataset_name, batch_size, tokenizer)

# Preparing components for Accelerate
    model, optimizer, train_loader, eval_loader = accelerator.prepare(model, optimizer, train_loader, eval_loader)

# Fitting the Model
    for epoch in range(1, epochs+1):  # Training for 3 epochs
        train_accuracy, train_loss = train(epoch, model, train_loader, optimizer, accelerator, metric)
        validation_accuracy, validation_loss = evaluate(model, eval_loader, metric)

        if accelerator.is_local_main_process:
            print(f"Training Accuracy: {train_accuracy:.4f}, Training Loss: {train_loss:.4f}")
            print(f"Validation Accuracy: {validation_accuracy:.4f}, Validation Loss: {validation_loss:.4f}")

if __name__ == "__main__":
    from accelerate import notebook_launcher
    notebook_launcher(main, num_processes=2, mixed_precision="fp16")

In [ ]:
file_content="""
import torch
from evaluate import load
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from accelerate import Accelerator
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def compute_metric(predictions, references, total_loss, loader, metric, custom="no"):
    if custom=="yes":
        print("hello")
    else:
        metrics_result = metric.compute(predictions=predictions, references=references)
        metrics = metrics_result['accuracy']
        loss = total_loss / len(loader)
        return metrics, loss


def dataset(dataset_name, batch_size, tokenizer):
    dataset = load_dataset(dataset_name, 'mrpc')

    def encode(examples):
        return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length', max_length=128)

    dataset = dataset.map(encode, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    train_dataset = dataset['train']
    eval_dataset = dataset['validation']

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=data_collator)

    return train_loader, eval_loader


def train(epoch, model, loader, optimizer, accelerator, metric):
    model.train()
    total_loss = 0
    all_predictions = []
    all_references = []
    loop = tqdm(loader, desc=f"Training Epoch {epoch}", disable=not accelerator.is_local_main_process)
    for batch in loop:
        outputs = model(**batch)
        loss = outputs.loss
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        references = batch['labels'].detach().cpu().numpy()
        all_predictions.extend(predictions)
        all_references.extend(references)

    # Compute accuracy using evaluate
    train_accuracy, train_loss = compute_metric(all_predictions, all_references, total_loss, loader, metric)
    return train_accuracy, train_loss



# Evaluation function without tqdm
def evaluate(model, loader, metric):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_references = []
    for batch in loader:
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
            loss = outputs.loss
            total_loss += loss.item()

            predictions = torch.argmax(logits, dim=-1).detach().cpu().numpy()
            references = batch['labels'].detach().cpu().numpy()
            all_predictions.extend(predictions)
            all_references.extend(references)

    # Compute accuracy using evaluate
    valid_accuracy, valid_loss = compute_metric(all_predictions, all_references, total_loss, loader, metric)
    return valid_accuracy, valid_loss


if __name__ == "__main__":
# Initiate Accelerator
    accelerator = Accelerator()

# Download Model and Tokenizer
    checkpoint = 'bert-base-uncased'
    model = BertForSequenceClassification.from_pretrained(checkpoint, force_download=True, num_labels=2)
    tokenizer = BertTokenizer.from_pretrained(checkpoint, force_download=True)

# Setting up Hyperparameters
    epochs = 10
    lr = 5e-4
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    batch_size = 128
    metric = load("accuracy")

# Processing Dataset
    dataset_name = 'glue'
    train_loader, eval_loader = dataset(dataset_name, batch_size, tokenizer)

# Preparing components for Accelerate
    model, optimizer, train_loader, eval_loader = accelerator.prepare(model, optimizer, train_loader, eval_loader)

# Fitting the Model
    for epoch in range(1, epochs+1):  # Training for 3 epochs
        train_accuracy, train_loss = train(epoch, model, train_loader, optimizer, accelerator, metric)
        validation_accuracy, validation_loss = evaluate(model, eval_loader, metric)

        if accelerator.is_local_main_process:
            print(f"Training Accuracy: {train_accuracy:.4f}, Training Loss: {train_loss:.4f}")
            print(f"Validation Accuracy: {validation_accuracy:.4f}, Validation Loss: {validation_loss:.4f}")
"""


# Create and write to the file in the /kaggle/working/ directory
file_path = "/kaggle/working/train_script.py"
with open(file_path, "w") as file:
    file.write(file_content)

print("File created successfully in /kaggle/working/")

In [ ]:
# !accelerate launch --multi_gpu --num_processes=2 train_script.py
!accelerate launch --multi_gpu --num_processes=2 --mixed_precision=fp16 train_script.py
# !accelerate launch --multi_gpu --mixed_precision=fp16 --num_processes=2 train_script.py


In [ ]:
from evaluate import load
# evaluate.list_evaluation_modules()
predictions = [1, 0, 0 ,1]
references = [0,0,0,1]
metric = load("accuracy")
accuracy = metric.compute(predictions=predictions, references=references)
accuracy

In [ ]:
!accelerate config

In [ ]:
!accelerate launch -h

In [ ]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, DataCollatorWithPadding
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from accelerate import Accelerator

In [ ]:
# Define the content you want to write to the file
file_content = """
def hello_world():
    print("Hello, World!")

if __name__=="__main__":
    hello_world()
"""

# Create and write to the file in the /kaggle/working/ directory
file_path = "/kaggle/working/example.py"
with open(file_path, "w") as file:
    file.write(file_content)

print("File created successfully in /kaggle/working/")


In [ ]:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_devices = torch.cuda.device_count()
    print(f'Using device: {device}')
    print(f'Number of available devices: {num_devices}')